In [1]:
import numpy as np
content = np.loadtxt('/content/P1_5.txt', dtype=str)
print(content)

[['1' '0' '6.77864488255' ... '0.742605144742' '0.347446160889'
  '0.131211251856']
 ['1' '1' '-0.354725173723' ... '2.01310088506' '1.28743035823'
  '0.521263818116']
 ['1' '1' '1.72771550354' ... '0.288585321447' '0.67945383572'
  '0.822154845116']
 ...
 ['2' '0' '2.0742480563' ... '1.18131645707' '1.38569938441'
  '0.999760836326']
 ['2' '1' '-0.0564591439281' ... '0.742153560433' '0.124074097398'
  '-0.0904859280961']
 ['2' '1' '-0.0483328162199' ... '1.04595614066' '0.737970144712'
  '0.831975723135']]


Ejercicio 1

1.1

In [2]:
# Extraer la primera columna que corresponde a las clases
clases = content[:, 0]

# Contar la frecuencia de cada clase y calcular su proporción
clase_1_count = np.sum(clases == '1')
clase_2_count = np.sum(clases == '2')
total = len(clases)
proporcion_clase_1 = clase_1_count / total
proporcion_clase_2 = clase_2_count / total

print(f"Clase 1 (Atención): {clase_1_count} instancias ({proporcion_clase_1:.2%})")
print(f"Clase 2 (Sin Atención): {clase_2_count} instancias ({proporcion_clase_2:.2%})")


Clase 1 (Atención): 281 instancias (14.26%)
Clase 2 (Sin Atención): 1689 instancias (85.74%)


Observando lo anterior podemos ver que los datos no están balanceados. Por lo tanto, aplicaré una estrategia de submuestreo de los datos mayoritarios para mitigar el hecho de que la muestra no esté balanceada.

In [ ]:
"""

import numpy as np

clase_1_indices = np.where(content[:, 0] == '1')[0]
clase_2_indices = np.where(content[:, 0] == '2')[0]

# Submuestreo de la clase 2 (mayoritaria)
np.random.seed(281)  # Número de instancias de la clase 1
clase_2_submuestreada_indices = np.random.choice(
    clase_2_indices, size=len(clase_1_indices), replace=False)

# Combinar los índices submuestreados de la clase 2 con todos los índices de la clase 1
indices_balanceados = np.concatenate([clase_1_indices, clase_2_submuestreada_indices])
np.random.shuffle(indices_balanceados)
datos_balanceados = content[indices_balanceados]

print("Primeras filas del conjunto de datos balanceado:")
print(datos_balanceados[:10])

"""

Primeras filas del conjunto de datos balanceado:
[['2' '1' '-1.32031574568' ... '0.690991417864' '0.934780691239'
  '1.22825612175']
 ['2' '1' '0.715251473588' ... '0.768991642588' '0.130260394012'
  '0.211868715382']
 ['2' '1' '-1.59027205088' ... '2.34066981864' '1.84007890083'
  '0.618981722881']
 ...
 ['2' '1' '-2.1078340269' ... '1.8729579623' '2.00573682622'
  '1.39843651386']
 ['1' '1' '-0.367503011048' ... '0.21162335027' '0.199326074091'
  '-0.538510096287']
 ['1' '1' '-1.5866503046' ... '1.14755695051' '1.04392262927'
  '0.845145512013']]


1.2

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

X = content[:, 2:].astype(float)
y = content[:, 0].astype(int)




In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample

models = [
    ('Logistic Regression', LogisticRegression(max_iter=10000)),
    ('Support Vector Classifier', SVC()),
    ('Random Forest Classifier', RandomForestClassifier()),
    ('Gradient Boosting Classifier', GradientBoostingClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('Decision Tree Classifier', DecisionTreeClassifier()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('AdaBoost Classifier', AdaBoostClassifier())
]

results = {}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models:
    scores = []
    for train_index, test_index in skf.split(X, y):
        # Submuestreo dentro de cada pliegue
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Submuestreo de la clase mayoritaria
        X_train_majority = X_train_fold[y_train_fold == 2]
        y_train_majority = y_train_fold[y_train_fold == 2]
        X_train_minority = X_train_fold[y_train_fold == 1]
        y_train_minority = y_train_fold[y_train_fold == 1]

        X_train_majority_downsampled, y_train_majority_downsampled = resample(
            X_train_majority, y_train_majority,
            replace=False,  # sin reemplazo
            n_samples=len(y_train_minority),  # igualar el tamaño de la clase minoritaria
            random_state=42
        )

        # Crear un nuevo conjunto de entrenamiento balanceado
        X_train_balanced = np.vstack((X_train_minority, X_train_majority_downsampled))
        y_train_balanced = np.concatenate((y_train_minority, y_train_majority_downsampled))

        # Crear un pipeline para estandarizar los datos y aplicar el modelo
        pipeline = make_pipeline(StandardScaler(), model)
        pipeline.fit(X_train_balanced, y_train_balanced)
        score = pipeline.score(X_test_fold, y_test_fold)
        scores.append(score)

    results[name] = np.mean(scores)

for name, score in results.items():
    print(f"{name}: {score:.2f}")


Logistic Regression: 0.84
Support Vector Classifier: 0.85
Random Forest Classifier: 0.84
Gradient Boosting Classifier: 0.83
K-Nearest Neighbors: 0.67
Naive Bayes: 0.78
Decision Tree Classifier: 0.71
Quadratic Discriminant Analysis: 0.58
AdaBoost Classifier: 0.82


El modelo más efectivo es el support vector classifier.

1.3

In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(y, y_pred):
    m = len(y)
    cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    return cost

def gradient_descent(X, y, learning_rate, epochs):
    m, n = X.shape
    theta = np.zeros(n)
    costs = []

    for _ in range(epochs):
        z = np.dot(X, theta)
        y_pred = sigmoid(z)
        gradient = (1/m) * np.dot(X.T, (y_pred - y))
        theta -= learning_rate * gradient
        cost = compute_cost(y, y_pred)
        costs.append(cost)

    return theta, costs

def logistic_regression(X_train, y_train, X_test, y_test, learning_rate=0.01, epochs=1000):
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    X_test = np.c_[np.ones(X_test.shape[0]), X_test]

    theta, costs = gradient_descent(X_train, y_train, learning_rate, epochs)
    y_pred_prob = sigmoid(np.dot(X_test, theta))
    y_pred = (y_pred_prob >= 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy, theta, costs

X = content[:, 2:].astype(float)
y = content[:, 0].astype(int)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Submuestreo de la clase mayoritaria
    X_train_majority = X_train_fold[y_train_fold == 2]
    y_train_majority = y_train_fold[y_train_fold == 2]
    X_train_minority = X_train_fold[y_train_fold == 1]
    y_train_minority = y_train_fold[y_train_fold == 1]

    X_train_majority_downsampled, y_train_majority_downsampled = resample(
        X_train_majority, y_train_majority,
        replace=False,
        n_samples=len(y_train_minority),
        random_state=42
    )

    X_train_balanced = np.vstack((X_train_minority, X_train_majority_downsampled))
    y_train_balanced = np.concatenate((y_train_minority, y_train_majority_downsampled))

    scaler = StandardScaler()
    X_train_balanced = scaler.fit_transform(X_train_balanced)
    X_test_fold = scaler.transform(X_test_fold)

    accuracy, theta, costs = logistic_regression(X_train_balanced, y_train_balanced, X_test_fold, y_test_fold)
    accuracies.append(accuracy)

print(f'Precisión promedio del modelo de Regresión Logística: {np.mean(accuracies):.2f}')



# Referencia https://www.geeksforgeeks.org/understanding-logistic-regression/

<ipython-input-7-81159b6b8d0c>:14: RuntimeWarning: divide by zero encountered in log
  cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
<ipython-input-7-81159b6b8d0c>:14: RuntimeWarning: divide by zero encountered in log
  cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
<ipython-input-7-81159b6b8d0c>:14: RuntimeWarning: divide by zero encountered in log
  cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))


Precisión promedio del modelo de Regresión Logística: 0.11


1.4

In [9]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

X = content[:, 2:].astype(float)
y = content[:, 0].astype(int)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def feature_selection_and_evaluation(X, y, max_features):
    best_accuracy = 0
    best_num_features = 0
    all_accuracies = []

    for k in range(1, max_features + 1):
        fold_accuracies = []

        for train_index, test_index in skf.split(X, y):
            X_train_fold, X_test_fold = X[train_index], X[test_index]
            y_train_fold, y_test_fold = y[train_index], y[test_index]

            X_train_majority = X_train_fold[y_train_fold == 2]
            y_train_majority = y_train_fold[y_train_fold == 2]
            X_train_minority = X_train_fold[y_train_fold == 1]
            y_train_minority = y_train_fold[y_train_fold == 1]

            X_train_majority_downsampled, y_train_majority_downsampled = resample(
                X_train_majority, y_train_majority,
                replace=False,
                n_samples=len(y_train_minority),
                random_state=42
            )

            X_train_balanced = np.vstack((X_train_minority, X_train_majority_downsampled))
            y_train_balanced = np.concatenate((y_train_minority, y_train_majority_downsampled))

            scaler = StandardScaler()
            X_train_balanced = scaler.fit_transform(X_train_balanced)
            X_test_fold = scaler.transform(X_test_fold)

            selector = SelectKBest(f_classif, k=k)
            X_train_selected = selector.fit_transform(X_train_balanced, y_train_balanced)
            X_test_selected = selector.transform(X_test_fold)

            model = SVC()
            model.fit(X_train_selected, y_train_balanced)
            y_pred = model.predict(X_test_selected)
            accuracy = accuracy_score(y_test_fold, y_pred)
            fold_accuracies.append(accuracy)

        mean_accuracy = np.mean(fold_accuracies)
        all_accuracies.append(mean_accuracy)

        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_num_features = k

    return best_num_features, best_accuracy, all_accuracies

max_features = X.shape[1]
best_num_features, best_accuracy, all_accuracies = feature_selection_and_evaluation(X, y, max_features)

print(f'Número óptimo de características: {best_num_features}')
print(f'Precisión del modelo con el número óptimo de características: {best_accuracy:.2f}')


Número óptimo de características: 141
Precisión del modelo con el número óptimo de características: 0.86


1.5

In [ ]:
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Usando una muestra más pequeña de los datos balanceados para acelerar el proceso (NO CORRÍA SI NO AGREGABA ESTE CÓDIGO)
sample_indices = np.random.choice(X.shape[0], size=int(X.shape[0] * 0.1), replace=False)
X_sample = X[sample_indices]
y_sample = y[sample_indices]

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svc = SVC(kernel='linear')  # Usar un kernel lineal para acelerar el cálculo
sfs = SequentialFeatureSelector(svc, n_features_to_select='auto', direction='forward', scoring='accuracy', cv=5)

# Crear un pipeline con estandarización y SFS
pipeline = Pipeline([
    ('selector', sfs),
    ('classifier', svc)
])

pipeline.fit(X_train, y_train)
selected_features = sfs.get_support()
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Número de características seleccionadas: {np.sum(selected_features)}')
print(f'Características seleccionadas: {np.where(selected_features)[0]}')
print(f'Precisión del modelo con características seleccionadas: {accuracy:.2f}')



Número de características seleccionadas: 76
Características seleccionadas: [  0   1   2   6   7   8   9  12  13  14  15  16  17  20  21  22  26  27
  31  33  34  35  36  39  40  41  42  43  44  45  46  47  48  49  51  52
  53  55  56  57  58  59  60  61  64  68  73  74  75  80  81  82  83  84
  85  86  88  89  90 102 103 104 105 110 117 118 119 120 121 122 141 142
 144 145 146 150]
Precisión del modelo con características seleccionadas: 0.71


1.6

In [8]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV

accuracies = []
selected_features_count = []
feature_ranking = []

for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Submuestreo de la clase mayoritaria
    X_train_majority = X_train_fold[y_train_fold == 2]
    y_train_majority = y_train_fold[y_train_fold == 2]
    X_train_minority = X_train_fold[y_train_fold == 1]
    y_train_minority = y_train_fold[y_train_fold == 1]

    X_train_majority_downsampled, y_train_majority_downsampled = resample(
        X_train_majority, y_train_majority,
        replace=False,
        n_samples=len(y_train_minority),
        random_state=42
    )

    X_train_balanced = np.vstack((X_train_minority, X_train_majority_downsampled))
    y_train_balanced = np.concatenate((y_train_minority, y_train_majority_downsampled))

    scaler = StandardScaler()
    X_train_balanced = scaler.fit_transform(X_train_balanced)
    X_test_fold = scaler.transform(X_test_fold)

    svc = SVC(kernel='linear')
    rfecv = RFECV(estimator=svc, step=1, cv=5, scoring='accuracy')
    rfecv.fit(X_train_balanced, y_train_balanced)

    y_pred = rfecv.predict(X_test_fold)
    accuracy = accuracy_score(y_test_fold, y_pred)
    accuracies.append(accuracy)
    selected_features_count.append(rfecv.n_features_)
    feature_ranking.append(rfecv.ranking_)

print(f'Precisión promedio del modelo con características seleccionadas: {np.mean(accuracies):.2f}')
print(f'Número promedio óptimo de características: {np.mean(selected_features_count):.2f}')
print(f'Ranking de características promedio: {np.mean(feature_ranking, axis=0)}')


Precisión promedio del modelo con características seleccionadas: 0.82
Número promedio óptimo de características: 38.40
Ranking de características promedio: [37.6 36.6 52.8 52.4 69.  64.6 27.8 22.2 10.2  1.   1.   2.6 73.2  4.6
  2.4 32.8 34.2 30.8 55.6 36.  44.8 42.4 48.2  3.   9.8 31.2 38.6 19.
 49.6 69.6 42.2  7.  57.  47.2 40.2 21.2 42.  67.  51.4  1.  38.8 63.8
 14.8 47.6 45.4 45.2 52.2 45.2 38.  49.8 27.6 47.4 53.6 65.6 47.6 40.
 48.6 50.  58.4 43.  57.8 61.4 46.8 50.2 97.6 26.   3.2 16.2 61.6 50.
 40.  76.6 82.4 52.4 51.4 60.4 76.2 28.4 35.   7.6 53.8 42.  53.  74.2
 81.8 67.4 45.2 76.4 64.2 43.4 20.2 29.4 59.8 60.8 71.8 59.8 87.  56.4
 35.2 53.8 41.2 31.2 29.6 46.4 69.  39.  44.2 38.4 46.4 49.2 59.6 29.
  8.4 61.8  8.6 38.2 80.4 81.4 52.  90.4 79.4 37.2  2.4 53.  43.  58.2
 80.4 62.6 10.6 65.2 24.  47.6 41.6 77.4 41.4 42.8 77.6 69.2 42.4 60.
 39.4 12.2 44.2 46.  67.4 68.2 44.8 41.4 47.  36.2 51.  55.8 34.8]


1.7 (preguntar diferencia con la anterior)

In [ ]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV

accuracies = []
selected_features_count = []
feature_ranking = []

for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Submuestreo de la clase mayoritaria
    X_train_majority = X_train_fold[y_train_fold == 2]
    y_train_majority = y_train_fold[y_train_fold == 2]
    X_train_minority = X_train_fold[y_train_fold == 1]
    y_train_minority = y_train_fold[y_train_fold == 1]

    X_train_majority_downsampled, y_train_majority_downsampled = resample(
        X_train_majority, y_train_majority,
        replace=False,
        n_samples=len(y_train_minority),
        random_state=42
    )

    X_train_balanced = np.vstack((X_train_minority, X_train_majority_downsampled))
    y_train_balanced = np.concatenate((y_train_minority, y_train_majority_downsampled))

    scaler = StandardScaler()
    X_train_balanced = scaler.fit_transform(X_train_balanced)
    X_test_fold = scaler.transform(X_test_fold)

    svc = SVC(kernel='linear')
    rfecv = RFECV(estimator=svc, step=1, cv=5, scoring='accuracy')
    rfecv.fit(X_train_balanced, y_train_balanced)

    y_pred = rfecv.predict(X_test_fold)
    accuracy = accuracy_score(y_test_fold, y_pred)
    accuracies.append(accuracy)
    selected_features_count.append(rfecv.n_features_)
    feature_ranking.append(rfecv.ranking_)

print(f'Precisión promedio del modelo con características seleccionadas: {np.mean(accuracies):.2f}')
print(f'Número promedio óptimo de características: {np.mean(selected_features_count):.2f}')
print(f'Ranking de características promedio: {np.mean(feature_ranking, axis=0)}')


Número óptimo de características: 26
Ranking de características: [ 21   1  15   1 113   1  10  13   1   1   1   1  48  11   1 126 125   7
  95  94  52  24  12   1  90  38   1  56   9 111   4  51 102  39   2  49
  50  61  25   1  26  31 101  40   1  64  65   1 100 124  17  89  60  59
   1  19  75 104   1  58  66 106  41  69  76  57   1  22  70  87  37 114
  81  86  85 110  72  97  84  23 107  63  98  67  96  68 105 119   1  62
   1 103  99   1  46  47  34  45  14  92  93 118  18 122 121 117 116 115
  30  88   1  29   1  73   1  82  83  74  16 120  36  35   1  91  20   1
 108 109   5  54   1  44  43  77  42  27  32  79  80 123  28   3 128  78
 112  55 127  53   1   8  33  71   6]


1.8

1. ¿Qué pasa si no se considera el problema de tener datos desbalanceados para este caso? ¿Por qué?
Los modelos no hubieran sido tan precisos que que hubieramos contado con una clara falta de isntancias de una de las clases.
2. De todos los clasificadores, ¿cuál o cuales consideras que son adecuados para los datos? ¿Qué propiedades tienen dichos modelos que los hacen apropiados para los datos? Argumenta tu respuesta.
El SVC es el modelo más adecuado para las clases. El SVC se beneficia de maximizar el margen entre las clases, cosa que funciona aprticularmente bien con este set de datos. También, el SVC es más efectivo en espacios de alta dimensión. Cosa que es efectiva aquí
3. ¿Es posibles reducir la dimensionalidad del problema sin perder rendimiento en el modelo? ¿Por qué?
Si, si se hubiera trabajado con un sobreajuste como técnica de balanceo, sin embargo como trabajé con una técnica de subajuste no serviría reducir la dimensionalidad del problema sin afectar el rendimiento del modelo.
4. ¿Qué método de selección de características consideras el más adecuado para este caso? ¿Por qué?
Creo que el recursivo es mejor ya que por la simple cantidad de características que selecciona, lo vuelve más acertado.
5. Si quisieras mejorar el rendimiento de tus modelos, ¿qué más se podría hacer?
Podríamos hacer una busqueda y optimización de hiperparámetros para hacer el modelo más efectivo.

Ejercicio 2

2.1

In [ ]:
import numpy as np
content = np.loadtxt('/content/M_3.txt', dtype=str)
print(content)

[['1' '1' '-1.55867656055' ... '0.292100379445' '-0.435294126191'
  '1.3840817591']
 ['1' '1' '-1.97820748243' ... '0.134605372657' '-0.663639058713'
  '1.23454478775']
 ['1' '1' '-2.00252078572' ... '0.207432345973' '-0.563342605951'
  '1.04644537775']
 ...
 ['7' '1' '-4.9445669698' ... '-1.32610563077' '-0.601560252125'
  '-5.29772692703']
 ['7' '1' '-3.65328964328' ... '-1.68888044294' '-0.297471104631'
  '-5.37944000078']
 ['7' '1' '-4.66665759595' ... '-1.74725351068' '-0.371247669527'
  '-5.16867605899']]


In [ ]:
# Extraer la primera columna que corresponde a las clases
clases = content[:, 0]

# Contar la frecuencia de cada clase y calcular su proporción
clase_1_count = np.sum(clases == '1')
clase_2_count = np.sum(clases == '2')
clase_3_count = np.sum(clases == '3')
clase_4_count = np.sum(clases == '4')
clase_5_count = np.sum(clases == '5')
clase_6_count = np.sum(clases == '6')
clase_7_count = np.sum(clases == '7')

total = len(clases)

proporcion_clase_1 = clase_1_count / total
proporcion_clase_2 = clase_2_count / total
proporcion_clase_3 = clase_3_count / total
proporcion_clase_4 = clase_4_count / total
proporcion_clase_5 = clase_5_count / total
proporcion_clase_6 = clase_6_count / total
proporcion_clase_7 = clase_7_count / total

print(f"Clase 1: {clase_1_count} instancias ({proporcion_clase_1:.2%})")
print(f"Clase 2: {clase_2_count} instancias ({proporcion_clase_2:.2%})")
print(f"Clase 3: {clase_3_count} instancias ({proporcion_clase_3:.2%})")
print(f"Clase 4: {clase_4_count} instancias ({proporcion_clase_4:.2%})")
print(f"Clase 5: {clase_5_count} instancias ({proporcion_clase_5:.2%})")
print(f"Clase 6: {clase_6_count} instancias ({proporcion_clase_6:.2%})")
print(f"Clase 7: {clase_6_count} instancias ({proporcion_clase_7:.2%})")

Clase 1: 90 instancias (14.29%)
Clase 2: 90 instancias (14.29%)
Clase 3: 90 instancias (14.29%)
Clase 4: 90 instancias (14.29%)
Clase 5: 90 instancias (14.29%)
Clase 6: 90 instancias (14.29%)
Clase 7: 90 instancias (14.29%)


In [ ]:
"""
import numpy as np

clase_1_indices = np.where(content[:, 0] == '1')[0]
clase_2_indices = np.where(content[:, 0] == '2')[0]

# Submuestreo de la clase 2 (mayoritaria)
np.random.seed(281)  # Número de instancias de la clase 1
clase_2_submuestreada_indices = np.random.choice(
    clase_2_indices, size=len(clase_1_indices), replace=False)

# Combinar los índices submuestreados de la clase 2 con todos los índices de la clase 1
indices_balanceados = np.concatenate([clase_1_indices, clase_2_submuestreada_indices])
np.random.shuffle(indices_balanceados)
datos_balanceados = content[indices_balanceados]

print("Primeras filas del conjunto de datos balanceado:")
print(datos_balanceados[:10])
"""

Primeras filas del conjunto de datos balanceado:
[['2' '1' '-2.6042329146' ... '-0.677647168021' '-0.943000508497'
  '0.899909677596']
 ['2' '1' '-2.70038028644' ... '-0.639146593046' '-0.240777444659'
  '-0.435886545211']
 ['2' '1' '-2.79801739582' ... '-1.22207511602' '-0.69849756082'
  '0.0320692835053']
 ...
 ['1' '1' '-1.22710571275' ... '-0.188017608013' '-0.579827383565'
  '1.0976972883']
 ['2' '1' '-1.84870482199' ... '-1.07974079532' '0.223656204654'
  '0.854048882835']
 ['1' '1' '-0.84458637736' ... '1.40722591425' '0.299096588313'
  '0.977341922083']]


Observamos que los datos ya están balanceados, por lo que vamos a trabajar con el df original.

2.2

In [ ]:
X = content[:, 2:]  # Ignoramos la segunda columna y tomamos el resto como características
y = content[:, 0].astype(int)  # La primera columna es la variable objetivo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


models = [
    ('Logistic Regression', LogisticRegression(max_iter=10000)),
    ('Support Vector Classifier', SVC()),
    ('Random Forest Classifier', RandomForestClassifier()),
    ('Gradient Boosting Classifier', GradientBoostingClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('Decision Tree Classifier', DecisionTreeClassifier()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('AdaBoost Classifier', AdaBoostClassifier())
]

results = {}
for name, model in models:
    # Crear un pipeline para estandarizar los datos y aplicar el modelo
    pipeline = make_pipeline(StandardScaler(), model)
    # Realizar validación cruzada y calcular la precisión promedio
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    results[name] = cv_scores.mean()

for name, score in results.items():
    print(f"{name}: {score:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Logistic Regression: 0.97
Support Vector Classifier: 0.95
Random Forest Classifier: 0.93
Gradient Boosting Classifier: 0.90
K-Nearest Neighbors: 0.93
Naive Bayes: 0.86
Decision Tree Classifier: 0.79
Quadratic Discriminant Analysis: 0.27
AdaBoost Classifier: 0.42


2.3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

X = content[:, 2:]
y = content[:, 0].astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Parameter grid for Decision Tree Classifier
param_grid_dt = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Parameter grid for SVC
param_grid_svc = {
    'C': np.logspace(-3, 2, 6),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 1, 5))
}

# Setup the models with GridSearchCV
gs_dt = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid_dt, cv=cv_inner, scoring='accuracy')
gs_svc = GridSearchCV(estimator=SVC(), param_grid=param_grid_svc, cv=cv_inner, scoring='accuracy')

models = {
    'SVC': gs_svc,
    'Decision Tree': gs_dt
}

for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, scoring='accuracy', cv=cv_outer)
    print(f"{name}: Mean Accuracy: {np.mean(scores):.2f} (+/- {np.std(scores)*2:.2f})")

    model.fit(X_scaled, y)
    print(f"Best hyperparameters for {name}: {model.best_params_}")


SVC: Mean Accuracy: 0.97 (+/- 0.02)
Best hyperparameters for SVC: {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Decision Tree: Mean Accuracy: 0.81 (+/- 0.03)
Best hyperparameters for Decision Tree: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}


2.4

Preguntar diferencia con la anterior

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Suponiendo que 'content' contiene los datos cargados
X = content[:, 2:]  # Ignoramos la segunda columna y tomamos el resto como características
y = content[:, 0].astype(int)  # La primera columna es la variable objetivo

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir la estrategia de validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definir los grids de hiperparámetros
param_grid_dt = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_svc = {
    'C': np.logspace(-3, 2, 6),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 1, 5))
}

# Configurar y ajustar GridSearchCV
model_dt = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid_dt, cv=cv, scoring='accuracy')
model_svc = GridSearchCV(SVC(), param_grid=param_grid_svc, cv=cv, scoring='accuracy')

# Ajustar los modelos
model_dt.fit(X_scaled, y)
model_svc.fit(X_scaled, y)


print("Mejores hiperparámetros para Decision Tree:", model_dt.best_params_)
print("Mejores hiperparámetros para SVC:", model_svc.best_params_)

final_model_dt = DecisionTreeClassifier(**model_dt.best_params_)
final_model_svc = SVC(**model_svc.best_params_)

final_model_dt.fit(X_scaled, y)
final_model_svc.fit(X_scaled, y)

print("Modelos ajustados y listos para producción.")


Mejores hiperparámetros para Decision Tree: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mejores hiperparámetros para SVC: {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Modelos ajustados y listos para producción.


2.5

Contesta lo siguientes:
1. ¿Observas un problema en cuanto al balanceo de las clases? ¿Por qué?
En este set de datos que me tocó no fue necesario hacer un balanceo de los datos ya que las 7 clases contaban con la misma cantidad de instancias.
2. ¿Qué modelo o modelos fueron efectivos para clasificar tus datos? ¿Observas algo especial sobre los modelos? Argumenta tu respuesta.
La regresión logísitca y el SVC fueron los modelos más efectivos para clasificar mis datos. Algo que veo es que en ambos ejercicios el SVC fue de los más efectivos en todos momentos.
3. ¿Observas alguna mejora importante al optimizar hiperparámetros? ¿Es el resultado que esperabas? Argumenta tu respuesta.
Si, los modelos se vuelven ligeramente más precisos, exactamente un 0.02% mejores cada uno. Si esperaba eso ya que al encontrar los hiperparámetros podríamos trabajar con una slección muy conveniente de hiperparámetros.
4. ¿Qué inconvenientes hay al encontrar hiperparámetros? ¿Por qué?
Realmente el único inconveniente que experimenté al encontrar los hiperparámetros fue el costo computacional de este mismo. El código tardo más de un minuto en correr en ambos casos.